In [9]:
# --- Cell 1 : interactive inputs ---------------------------------
import ipywidgets as widgets
from ipywidgets import interact
from datetime import date as _date

lat_widget  = widgets.FloatText(value=-27.47711,
                                description='Latitude (°)',
                                step=0.0001)
lon_widget  = widgets.FloatText(value=153.029342,
                                description='Longitude (°)',
                                step=0.0001)
date_widget = widgets.DatePicker(description='Date',
                                 value=_date(2025, 8, 4))

print("Enter your observing location and date, then run the next cell.")
display(lat_widget, lon_widget, date_widget)

Enter your observing location and date, then run the next cell.


FloatText(value=-27.47711, description='Latitude (°)', step=0.0001)

FloatText(value=153.029342, description='Longitude (°)', step=0.0001)

DatePicker(value=datetime.date(2025, 8, 4), description='Date')

In [8]:
# --- Cell 2 : calculation & LaTeX table --------------------------
import pandas as pd
from datetime import datetime, time, timedelta
import pytz
from astropy.coordinates import EarthLocation, get_sun, AltAz
from astropy.time import Time
import astropy.units as u

# Read values from widgets
latitude  = lat_widget.value
longitude = lon_widget.value
calculation_date = date_widget.value

TIMEZONE = pytz.timezone('Australia/Brisbane')  # adjust if desired

def calculate_solar_elevation(lat, lon, date_of_interest):
    location = EarthLocation(lat=lat*u.deg, lon=lon*u.deg, height=0*u.m)
    results = []
    start_time = time(9, 0)
    end_time   = time(15, 0)
    current    = datetime.combine(date_of_interest, start_time)
    interval   = timedelta(minutes=30)

    while current.time() <= end_time:
        dt_aware = TIMEZONE.localize(current)
        t = Time(dt_aware)
        sun_altaz = get_sun(t).transform_to(AltAz(obstime=t,
                                                 location=location))
        results.append({
            "Time (AEST)": current.strftime('%H:%M'),
            "Elevation (°)": f"{sun_altaz.alt.degree:.1f}"
        })
        current += interval

    return pd.DataFrame(results)

elevation_table = calculate_solar_elevation(latitude, longitude,
                                            calculation_date)

print(f"Solar Elevation for ({latitude:.4f}, {longitude:.4f})")
print(f"Date: {calculation_date.strftime('%d %B %Y')}")
print("-" * 50)
print(elevation_table.to_string(index=False))

# I recommend confirming this against https://gml.noaa.gov/grad/solcalc/

Solar Elevation for (-27.4771, 153.0293)
Date: 05 August 2025
--------------------------------------------------
Time (AEST) Elevation (°)
      09:00          28.8
      09:30          33.5
      10:00          37.7
      10:30          41.1
      11:00          43.7
      11:30          45.2
      12:00          45.6
      12:30          44.7
      13:00          42.8
      13:30          39.9
      14:00          36.1
      14:30          31.7
      15:00          26.7
